# NeuralMF
* See the corresponding file in `../../TrainingAlphas` for more details

In [ ]:
source = "NeuralMF";

In [ ]:
import NBInclude: @nbinclude
@nbinclude("../Alpha.ipynb")
@nbinclude("../../TrainingAlphas/Neural/NeuralNetworkBase.ipynb");

## Data Preprocessing

In [ ]:
# Override methods in NeuralNetworkBase to use recommendee splits

@memoize LRU{Any,Any}(maxsize = 2) function get_epoch_outputs(split, implicit, num_users)
    @assert split == "training"
    sparse(get_recommendee_split(implicit))
end

@memoize LRU{Any,Any}(maxsize = 2) function get_epoch_residuals(
    split,
    residual_alphas,
    implicit,
    num_users,
)
    @assert split == "training"
    sparse(read_recommendee_alpha(residual_alphas, implicit))
end

@memoize LRU{Any,Any}(maxsize = 2) function get_epoch_weights(
    split,
    user_weight_decay,
    item_weight_decay,
    implicit,
    num_users,
)
    @assert split == "training"
    df = get_recommendee_split(implicit)
    user_counts = fill(length(df.rating), num_items())
    weights =
        expdecay(user_counts, user_weight_decay) .* expdecay(
            get_counts(split, implicit; by_item = true, per_rating = false),
            item_weight_decay,
        )

    sparse(RatingsDataset(df.user, df.item, weights[df.item]))
end;

## Retrain user embeddings

In [ ]:
function retrain_user_embeddings(params)
    # the weight initialization cares about the number of users,
    # so let's initialize assuming we have all users and then fine tune 
    # a single layer
    init_embedding = build_retrain_model(params["retrain_hyp"], params["m"])[1].weight[:, 1]
    hyp = params["retrain_hyp"]
    global G = @set hyp.num_users = 1
    m = build_retrain_model(G, params["m"])
    m[1].weight .= init_embedding
    m = m |> device
    Random.seed!(G.seed)
    ps = Flux.params(m[1])
    opt = get_optimizer(G.optimizer, G.learning_rate, G.regularization_params)
    # -1 epoch because the first call to stop! happens before training
    epochs = params["epochs"] - 1
    @showprogress for _ = 1:epochs
        train_epoch!(m, ps, opt)
        apply_zero_gradient!(m, ps, opt, true)
    end
    global G = nothing
    m |> cpu
end;

## Write alpha

In [ ]:
function compute_alpha(source)
    @info "computing alpha $source"
    params = read_params(source)
    m = retrain_user_embeddings(params)
    activation = params["hyp"].implicit ? softmax : identity
    preds = activation(m(1))
    write_recommendee_alpha(preds, source)
end;

In [ ]:
function compute_alpha()
    compute_alpha("NeuralImplicitUserItemBiases")
    compute_alpha("NeuralImplicitMatrixFactorization")
end;

In [ ]:
compute_alpha();